In [0]:
! uptime

 00:34:34 up 9 min,  0 users,  load average: 0.29, 0.10, 0.05


### Load tools

In [0]:
import geopy
import geopy.distance

In [0]:
import numpy as np

In [0]:
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from datetime import datetime

In [0]:
starttime = datetime.now()
starttime

datetime.datetime(2019, 7, 23, 0, 38, 2, 633713)

### Set parameters

In [0]:
# bounding box
area = 'ortigas_cbd'
# lat_ul, lon_ul, lat_lr, lon_lr = 14.638874, 121.065815, 14.633243, 121.071803 # xavierville
# lat_ul, lon_ul, lat_lr, lon_lr = 14.556876, 121.014324, 14.550963, 121.022297 # makati cbd old version
# lat_ul, lon_ul, lat_lr, lon_lr = 14.645881, 121.017012, 14.620093, 121.052964 # mplace
# lat_ul, lon_ul, lat_lr, lon_lr = 14.643861, 120.960233, 14.635147, 120.969497# srv
# lat_ul, lon_ul, lat_lr, lon_lr = 14.564829, 121.005092, 14.546877, 121.032082 # makati_cbd
lat_ul, lon_ul, lat_lr, lon_lr = 14.593224, 121.049180, 14.569895, 121.072943 # ortigas_cbd
lat_ul, lon_ul, lat_lr, lon_lr

(14.593224, 121.04918, 14.569895, 121.072943)

In [0]:
step_size = 128 # in meters

In [0]:
zoom = 216 # meters, eye altitude in GEarth

### Offset lonlat

In [0]:
length = geopy.distance.VincentyDistance((lat_lr, lon_ul), (lat_lr, lon_lr))
width = geopy.distance.VincentyDistance((lat_ul, lon_ul), (lat_lr, lon_ul))
length, width

(Distance(2.5607590880785995), Distance(2.581229943737145))

In [0]:
steps_east = int(np.ceil(length.m/step_size))
steps_south = int(np.ceil(width.m/step_size))
steps_east, steps_south

(21, 21)

In [0]:
def offset_lonlat(startlat, startlon, step_size, steps_x, steps_y):
  # Outputs geopoint of location offsetted from startpoint
  # Define starting point.
  start = geopy.Point(startlat, startlon)

  # Define a general distance object, initialized with a distance of 1 km.
  dy = geopy.distance.VincentyDistance(kilometers = steps_y * step_size/1000)
  dx = geopy.distance.VincentyDistance(kilometers = steps_x * step_size/1000)

  # Use the `destination` method with a bearing of 0 degrees (which is north)
  # in order to go from point `start` 1 km to north.
  # offset y
  desty = dy.destination(point=start, bearing=180) # south
  # offset x
  dest = dx.destination(point=desty, bearing=90) # east
  return dest#.format_decimal()

In [0]:
temp = offset_lonlat(lat_ul, lon_ul, step_size, 0, 1)
temp

Point(14.592067144843332, 121.04918000000002, 0.0)

### Write to dataframe

In [0]:
def filename(lat, lon, radius):
  return 'zm' + str(radius) + 'm_lat'+str(round(lat, 6)).replace('.', 'd') + '_' + 'lon'+str(round(lon, 6)).replace('.', 'd')

In [0]:
data = pd.DataFrame(columns = ['area', 'filename', 'lat', 'lon'])
data

,area,filename,lat,lon


In [0]:
temp = filename(lat_ul, lon_ul, step_size)
temp

'zm128m_lat14d593224_lon121d04918'

In [0]:
temp_row = pd.DataFrame([[1,2,3,4]], columns = data.columns)
data.append(temp_row)

,area,filename,lat,lon
0,1,2,3,4


In [0]:
for e in range(0, steps_east + 1):
  for s in range(0, steps_south + 1):
    # offset point
    temp_pt = offset_lonlat(lat_ul, lon_ul, step_size, e, s)
    temp_lat = temp_pt[0]
    temp_lon = temp_pt[1]
    # write to dataframe
    temp_name = filename(temp_lat, temp_lon, step_size)
    temp_row = pd.DataFrame([[area,temp_name,temp_lat,temp_lon]], columns = data.columns)
    data = data.append(temp_row)

data.to_csv('test_20190513.csv', index = False)

In [0]:
# f = open('test_20190513.txt', 'w')
# f.write(header)
# for e in range(0, steps_east + 1):
#   for s in range(0, steps_south + 1):
#     # offset point
#     temp_pt = offset_lonlat(lat_ul, lon_ul, step_size, e, s)
#     temp_lat = temp_pt[0]
#     temp_lon = temp_pt[1]
#     # write to dataframe
#     temp_name = filename(temp_lat, temp_lon, step_size)
#     temp_row = pd.DataFrame([[area,temp_name,temp_lat,temp_lon]], columns = data.columns)
#     data = data.append(temp_row)
#     # write to kml file
#     body = template.replace('xname', temp_name).replace('xlat', str(temp_lat)).replace('xlon', str(temp_lon)).replace('xzoom', str(zoom))
#     f.write(body)

# f.write(tail)
# f.close()
# data.to_csv('test_20190513.csv', index = False)

### Write to KML

In [0]:
header = """
<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">
<Document>
	<name>offset_test.kml</name>
	<StyleMap id="failed3">
		<Pair>
			<key>normal</key>
			<styleUrl>#failed72</styleUrl>
		</Pair>
		<Pair>
			<key>highlight</key>
			<styleUrl>#failed42</styleUrl>
		</Pair>
	</StyleMap>
	<StyleMap id="failed30">
		<Pair>
			<key>normal</key>
			<styleUrl>#failed7</styleUrl>
		</Pair>
		<Pair>
			<key>highlight</key>
			<styleUrl>#failed41</styleUrl>
		</Pair>
	</StyleMap>
	<StyleMap id="failed31">
		<Pair>
			<key>normal</key>
			<styleUrl>#failed70</styleUrl>
		</Pair>
		<Pair>
			<key>highlight</key>
			<styleUrl>#failed40</styleUrl>
		</Pair>
	</StyleMap>
	<Style id="failed4">
	</Style>
	<Style id="failed40">
	</Style>
	<StyleMap id="failed32">
		<Pair>
			<key>normal</key>
			<styleUrl>#failed71</styleUrl>
		</Pair>
		<Pair>
			<key>highlight</key>
			<styleUrl>#failed4</styleUrl>
		</Pair>
	</StyleMap>
	<Style id="failed7">
	</Style>
	<Style id="failed70">
	</Style>
	<Style id="failed41">
	</Style>
	<Style id="failed71">
	</Style>
	<Style id="failed72">
	</Style>
	<Style id="failed42">
	</Style>
	<Folder>
		<name>offset_test</name>
		<visibility>0</visibility>
		<open>1</open>
"""
header = header[1:]

In [0]:
template = """
		<Placemark>
			<name>xname</name>
			<visibility>0</visibility>
			<LookAt>
				<longitude>xlon</longitude>
				<latitude>xlat</latitude>
				<altitude>0</altitude>
				<heading>4.770832022195275e-015</heading>
				<tilt>0</tilt>
				<range>xzoom</range>
				<gx:altitudeMode>relativeToSeaFloor</gx:altitudeMode>
			</LookAt>
			<styleUrl>#failed3</styleUrl>
			<Point>
				<coordinates>xlon,xlat,0</coordinates>
			</Point>
		</Placemark>
"""
template = template[1:]

In [0]:
tail = """
	</Folder>
</Document>
</kml>
"""
tail = tail[1:]

In [0]:
body = ""
temp2 = template.replace('xname', temp)
body = body + temp2
body

'\t\t<Placemark>\n\t\t\t<name>zm128m_lat14d593224_lon121d04918</name>\n\t\t\t<visibility>0</visibility>\n\t\t\t<LookAt>\n\t\t\t\t<longitude>xlon</longitude>\n\t\t\t\t<latitude>xlat</latitude>\n\t\t\t\t<altitude>0</altitude>\n\t\t\t\t<heading>4.770832022195275e-015</heading>\n\t\t\t\t<tilt>0</tilt>\n\t\t\t\t<range>xzoom</range>\n\t\t\t\t<gx:altitudeMode>relativeToSeaFloor</gx:altitudeMode>\n\t\t\t</LookAt>\n\t\t\t<styleUrl>#failed3</styleUrl>\n\t\t\t<Point>\n\t\t\t\t<coordinates>xlon,xlat,0</coordinates>\n\t\t\t</Point>\n\t\t</Placemark>\n'

### Main routine

In [0]:
# # data = pd.read_csv('test_20190513.csv')
# data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/midpoints_cons2poly.csv')
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test_filtered.csv')
data.columns = ['area', 'filename', 'lat', 'lon']
data.head(2)

,area,filename,lat,lon
0,ortigas_cbd,zm128m_lat14d578185_lon121d051556,14.578185,121.051556
1,ortigas_cbd,zm128m_lat14d577028_lon121d051556,14.577028,121.051556


In [0]:
f = open('test_20190513.txt', 'w')
f.write(header)
for d in range(0,len(data)):
  # write to kml file
  row = data.iloc[d]
  body = template.replace('xname', row.filename).replace('xlat', str(row.lat)).replace('xlon', str(row.lon)).replace('xzoom', str(zoom))
  f.write(body)

f.write(tail)
f.close()

### Output

In [0]:
! ls

gdrive	sample_data  test_20190513.csv	test_20190513.txt


In [0]:
# data.head(2)

In [0]:
! cp test_20190513.csv "/content/gdrive/My Drive/Colab Notebooks/test.csv"

In [0]:
! cp test_20190513.txt "/content/gdrive/My Drive/Colab Notebooks/test.txt"

In [0]:
endtime = datetime.now()
duration = endtime - starttime

In [0]:
print('Processed ' + str((e + 1)*(s + 1)) + ' midpoints, took ' + str(round(duration.total_seconds()/60, 2)) + ' minutes.')

Processed 484 midpoints, took 10.36 minutes.
